In [1]:
!wget http://dlib.net/files/mmod_human_face_detector.dat.bz2
!bzip2 -d mmod_human_face_detector.dat.bz2

--2024-04-14 09:33:18--  http://dlib.net/files/mmod_human_face_detector.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694709 (678K)
Saving to: ‘mmod_human_face_detector.dat.bz2’

mmod_human_face_det 100%[===================>] 678.43K   460KB/s    in 1.5s    

2024-04-14 09:33:20 (460 KB/s) - ‘mmod_human_face_detector.dat.bz2’ saved [694709/694709]



In [2]:
import os
import shutil
import cv2 as cv
import dlib
import numpy as np
from skimage import io, color
import matplotlib.pyplot as plt

In [3]:
face_detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat")

In [5]:
from google.colab.patches import cv2_imshow

def crop_and_resize(img_name, output_img_name, face_detector, face_detector_lib='dlib'):
    img = io.imread(img_name)
    if img.ndim == 2:
        img = color.gray2rgb(img)
    elif img.shape[-1] == 4:
        img = img[..., :3]

    if face_detector_lib == 'dlib':
        detected_faces = face_detector(cv.cvtColor(img[..., ::-1].copy(), cv.COLOR_BGR2GRAY))

    face = detected_faces[0].rect

    test_img = cv.cvtColor(img.copy(), cv.COLOR_RGB2BGR)

    # Centrul fetei
    face_center_x = face.left() + (face.right() - face.left()) // 2
    face_center_y = face.top() + (face.bottom() - face.top()) // 2

    # Calculam top-left si bottom-right pentru ca sa obtinem o imagine
    # cu shape multiplu de (178, 218)
    width, height, _ = test_img.shape
    alpha = min(round(height / 178, 1), round(width / 218, 1))
    crop_width = round(178 * alpha)
    crop_height = round(218 * alpha)

    crop_x1 = max(0, face_center_x - crop_width // 2)
    crop_y1 = max(0, face_center_y - crop_height // 2)
    crop_x2 = min(test_img.shape[1], face_center_x + crop_width // 2)
    crop_y2 = min(test_img.shape[0], face_center_y + crop_height // 2)

    test_img = test_img[crop_y1:crop_y2, crop_x1:crop_x2]
    test_img = cv.resize(test_img, dsize=(178, 218),
                    interpolation=cv.INTER_LINEAR)

    final = cv.cvtColor(test_img.copy(), cv.COLOR_BGR2RGB)
    io.imsave(output_img_name, final)
    # cv2_imshow(final)

In [ ]:
# crop_and_resize(
#     img_name=f"/content/capture.png",
#     output_img_name=f"/content/4.jpg",
#     face_detector=face_detector)

In [ ]:
!unzip /content/data.zip

In [ ]:
import os
import shutil
import PIL.Image

def reorganize_yale_database(source_folder, destination_folder, subjects, conditions):
    for condition in conditions:
        os.makedirs(os.path.join(destination_folder, 'train', condition), exist_ok=True)
        os.makedirs(os.path.join(destination_folder, 'test', condition), exist_ok=True)

    for subject in subjects:
        for idx, condition in enumerate(conditions):
            source_path = os.path.join(source_folder, f"{subject}.{condition}")
            dest_folder = 'test' if subject.endswith('13') or subject.endswith('15') else 'train'
            dest_path = os.path.join(destination_folder, dest_folder, condition, f"{subject}_{condition}.jpg")
            shutil.copy(source_path, dest_path)
            cropped_image = crop_and_resize(dest_path, dest_path, face_detector=face_detector)

# data folder contains 165 images
source_folder = "/content/data"
# RaFD folder contains labeled folders with their specific items
destination_folder = "/content/RaFD/"
subjects = ['subject0' + str(nr) if nr < 10 else 'subject' + str(nr) for nr in range(1, 16)]
conditions = ['centerlight', 'happy', 'glasses', 'leftlight', 'noglasses', 'normal',
              'rightlight', 'sad', 'sleepy', 'surprised', 'wink']

reorganize_yale_database(source_folder, destination_folder, subjects, conditions)

In [10]:
from google.colab import drive
import shutil
import os

drive.mount('/content/drive')

source_folder = "/content/RaFD/"
destination_folder = "/content/drive/MyDrive/RaFD-178x218/"
shutil.copytree(source_folder, destination_folder)

print("Folder copied to Google Drive successfully.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Folder copied to Google Drive successfully.
